In [ ]:
!pip3 install -q -U pandas altair langcodes

In [ ]:
import sys
# Append system path
sys.path = [p for p in sys.path if not p.endswith("../..")]  # Cleans duplicated '../..'
sys.path.insert(0, "../")  # This adds `src` to the path
import os
import numpy as np
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows() # Allow using more than 5000 rows, for now
import langcodes
from helpers import io
from analysis import multimodal_util
from typing import Any


%load_ext autoreload
%autoreload 2

## Read Constants and Summaries

Load constants and data summaries from JSON files. Constants provide mappings and criteria for licenses, creator groups, various other categories. Data summaries contain modality-specific information about datasets.

- `all_constants`: Dictionary containing all predefined constants.
- `speech_summaries`: Data summaries for speech.

In [ ]:
all_constants = io.read_all_constants("../../constants/")
speech_summaries = io.read_data_summary_json("../../data_summaries-speech/")

license_paraphrases = multimodal_util.invert_dict_of_lists(all_constants["LICENSE_PARAPHRASES"])
speech_summaries = multimodal_util.map_license_criteria_multimodal(
    multimodal_util.remap_licenses_with_paraphrases(
        speech_summaries,
        license_paraphrases
    ),
    all_constants
)

df_speech = pd.DataFrame(speech_summaries)
df_speech, YEARS_ORDER = multimodal_util.factor_year(df_speech)

In [ ]:
# Overall Gini coefficient (hours by dataset)
multimodal_util.gini(df_speech["Hours"].values)

In [ ]:
# Plotting constants
FONT_SIZE = 20
LEGEND_POSITION = "bottom"
PLOT_TOFILE = True # Whether and where to output plots
PLOT_DIR = "~/Dropbox (MIT)/dpi-plotsspeech/"
PLOT_PPI = 300
MAX_LABELLIMIT = 1000 # Large number to avoid label summarization in plots

if PLOT_TOFILE:
    PLOT_DIR = os.path.expanduser(PLOT_DIR)
    os.makedirs(PLOT_DIR, exist_ok=True)

In [ ]:
def times_newroman():
    font = "Times New Roman"

    return {
          "config" : {
               "title": {"font": font},
               "axis": {
               "labelFont": font,
               "titleFont": font
          },
          "header": {
               "labelFont": font,
               "titleFont": font
          },
          "legend": {
               "labelFont": font,
               "titleFont": font
          },
          "text": {
               "font": font
          }
     }
}

alt.themes.register("times_newroman", times_newroman)
alt.themes.enable("times_newroman")

## License Use by Language Family and Source Category

In [ ]:
# Plotting constants
LICENSE_ORDER = ["Non-Commercial/Academic", "Unspecified", "Commercial"]
LICENSE_PALETTE = ["#e04c71", "#e0cd92", "#82b5cf"]
LICENSE_PLOTW = 600
LICENSE_PLOTH = 200

In [ ]:
# Map to main DPI license types
df_speech["License Type"] = df_speech["License Use (DataProvenance)"].map({
    "academic-only": "Non-Commercial/Academic",
    "non-commercial": "Non-Commercial/Academic",
    "unspecified": "Unspecified",
    "commercial": "Commercial"
})

df_speech["License Type"] = pd.Categorical(
    df_speech["License Type"],
    categories=LICENSE_ORDER,
    ordered=True
)
df_speech = df_speech.sort_values(by="License Type")

In [ ]:
# Remap language families for condensed plots
df_speechlanguages = df_speech.explode("Language Families")
df_speechlanguages["Language Families"] = df_speechlanguages["Language Families"].map(
    lambda x : {
        "English": "Germanic",
        "Hindustani": "Indo-European"
    }.get(x, x)
)

# Calculate permissiveness by language family (defined as the proportion of commercial licenses)
permisiveness = df_speechlanguages.groupby("Language Families").apply(
    lambda x: (x["License Type"] == "Commercial").mean()
).reset_index(name="Permissiveness")

# Sort by computed permisiveness
languagefamily_order = permisiveness.sort_values(by="Permissiveness")["Language Families"].tolist()

# Make factor
df_speechlanguages["Language Families"] = pd.Categorical(
    df_speechlanguages["Language Families"],
    categories=languagefamily_order,
    ordered=True
)

# Sort by language families
df_speechlanguages = df_speechlanguages.sort_values(by="Language Families")

In [ ]:
base = alt.Chart(df_speechlanguages).mark_bar().encode(
    x=alt.X(
        "Language Families:N",
        title="Language Family",
        sort=languagefamily_order,
        axis=alt.Axis(labelAngle=-30)
    ),
    y=alt.Y(
        "count():Q",
        stack="normalize",
        axis=alt.Axis(format="%"),
        title="Pct. Datasets"
    ),
    color=alt.Color(
        "License Type:N",
        scale=alt.Scale(
            domain=LICENSE_ORDER,
            range=LICENSE_PALETTE
        ),
        title="License Type"
    )
).properties(
    width=600,
    height=100
)

text = alt.Chart(df_speechlanguages).mark_text(
    dy=-68,
    align="center",
    baseline="top",
    fontSize=12
).encode(
    x=alt.X(
        "Language Families:N",
        title="Language Family",
        sort=languagefamily_order
    ),
    text="count():Q"
)

chart = (base + text).configure_axis(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE
).configure_legend(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    orient=LEGEND_POSITION,
    labelLimit=MAX_LABELLIMIT
)

if PLOT_TOFILE:
    chart.save(
        os.path.join(PLOT_DIR, "speech_languagefamilies-licenses.png"),
        ppi=PLOT_PPI
    )

chart

In [ ]:
INCLUDE_TOP_N_CATEGORIES = 10 # Number of top categories to include, rest will be grouped as "Other"

df_sources = df_speech.explode("Source Category")
df_sources = multimodal_util.reduce_categories_to_topk(df_sources, "Source Category", INCLUDE_TOP_N_CATEGORIES)

sourcecategory_order = multimodal_util.order_by_grouped_permisiveness(df_sources, "Source Category")

df_sources["Source Category"] = pd.Categorical(
    df_sources["Source Category"],
    categories=sourcecategory_order,
    ordered=True
)

df_sources = df_sources.sort_values(by="Source Category")

In [ ]:
base = alt.Chart(
    df_sources
).mark_bar().encode(
    x=alt.X(
        "Source Category:N",
        title="Source Category",
        axis=alt.Axis(labelAngle=-30),
        sort=sourcecategory_order
    ),
    y=alt.Y(
        "count():Q",
        stack="normalize",
        axis=alt.Axis(format="%"),
        title="Pct. Datasets"
    ),
    color=alt.Color(
        "License Type:N",
        scale=alt.Scale(
            domain=LICENSE_ORDER,
            range=LICENSE_PALETTE
        ),
        title="License Type"
    )
).properties(
    width=800,
    height=100
)

text = alt.Chart(df_sources).mark_text(
    dy=-68,
    align="center",
    baseline="top",
    fontSize=12
).encode(
    x=alt.X(
        "Source Category:N",
        title="Source Category",
        sort=sourcecategory_order
    ),
    text="count():Q"
)

chart = (base + text).configure_axis(
        labelFontSize=FONT_SIZE,
        titleFontSize=FONT_SIZE
).configure_legend(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    orient=LEGEND_POSITION,
    labelLimit=MAX_LABELLIMIT
).configure_header(
    titleFontSize=FONT_SIZE,
    labelFontSize=FONT_SIZE
)

if PLOT_TOFILE:
    chart.save(
        os.path.join(PLOT_DIR, "speech_sourcecategories-licenses.png"),
        ppi=PLOT_PPI
    )

chart

## Sources by Language Family

In [ ]:
INCLUDE_TOP_N_CATEGORIES = 6 # Number of top categories to include, rest will be grouped as "Other"

# # Further unlist the categories of sources
df_speechlanguagessources = multimodal_util.reduce_categories_to_topk(
    df_speechlanguages.explode("Source Category"),
    "Source Category",
    INCLUDE_TOP_N_CATEGORIES
)

In [ ]:
base = alt.Chart(
    df_speechlanguagessources
).mark_bar().encode(
    x=alt.X(
        "Language Families:N",
        title="Language Family",
        sort=languagefamily_order,
        axis=alt.Axis(labelAngle=-30)
    ),
    y=alt.Y(
        "count():Q",
        stack="normalize",
        axis=alt.Axis(format="%"),
        title="Pct. Datasets"
    ),
    color=alt.Color(
        "Source Category:N",
        title="Source Category"
    )
).properties(
    width=600,
    height=100
)

text = alt.Chart(df_speechlanguagessources).mark_text(
    dy=-68,
    align="center",
    baseline="top",
    fontSize=12
).encode(
    x=alt.X(
        "Language Families:N",
        title="Language Family",
        sort=languagefamily_order
    ),
    text="count():Q"
)

chart = (base + text).configure_axis(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE
).configure_legend(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    orient=LEGEND_POSITION,
    columns=4,
    labelLimit=MAX_LABELLIMIT
)

if PLOT_TOFILE:
    chart.save(
        os.path.join(PLOT_DIR, "speech_languagefamilies-sources.png"),
        ppi=PLOT_PPI
    )

chart

## Dataset Preprocessing for Languages
Retrieve precise hours per language from metadata for large-scale, highly multilingual datasets. Specifically: YODAS, Common Voice, Multilingual LibriSpeech, Bloom Speech, and FLEURS (for now).

In [ ]:
df_speechlanguagenames = df_speech.explode("Languages (ISO)")
df_speechlanguagenames["Language (Name)"] = df_speechlanguagenames["Languages (ISO)"].map(
    lambda x: langcodes.Language.make(language=langcodes.standardize_tag(x.split("-")[0].split("_")[0], macro=True)).language_name()
)

In [ ]:
df_yodashours = pd.read_csv("data/speech_supporting_data/yodas_splithours.csv").rename({"hours": "Hours"}, axis=1)
df_yodashours["Languages (ISO)"] = df_yodashours["split"].map(lambda x : x[:2])
df_yodashours["Language (Name)"] = df_yodashours["Languages (ISO)"].map(
    lambda x: langcodes.Language.make(language=langcodes.standardize_tag(x, macro=True)).language_name()
)

In [ ]:
df_commonvoicehours = pd.read_json("data/speech_supporting_data/commonvoice_splithours.json").T

language_codes_to_aggregate = {}
for langcode in df_commonvoicehours.index:
    if "-" in langcode or "_" in langcode:
        langcode_simplified = langcode.split("-")[0].split("_")[0]
        print("Will aggregate language %s to %s" % (langcode, langcode_simplified))
        language_codes_to_aggregate.setdefault(langcode_simplified, [])
        language_codes_to_aggregate[langcode_simplified].append(langcode)

for langcode_simplified, langcode_data_to_aggregate in language_codes_to_aggregate.items():
    df_commonvoicehours.loc[langcode_simplified, "total_clips_duration"] = df_commonvoicehours.loc[langcode_data_to_aggregate, "total_clips_duration"].sum()


df_commonvoicehours["Hours"] = df_commonvoicehours["total_clips_duration"] / 60 / 60 / 1000
df_commonvoicehours = df_commonvoicehours.reset_index(names=["Languages (ISO)"])

In [ ]:
df_multilinguallibrispeechhours = pd.read_csv("data/speech_supporting_data/multilinguallibrispeech_splithours.csv")
df_multilinguallibrispeechhours["Hours"] = df_multilinguallibrispeechhours[df_multilinguallibrispeechhours.columns[1:]].sum(axis=1)
df_multilinguallibrispeechhours["Languages (ISO)"] = df_multilinguallibrispeechhours.language.map(lambda x: langcodes.find(x).to_tag())

In [ ]:
df_bloomspeechhours = pd.read_csv("data/speech_supporting_data/bloomspeech_splithours.csv")
df_bloomspeechhours["Hours"] = df_bloomspeechhours[df_bloomspeechhours.columns[2:]].sum(axis=1) / 60
df_bloomspeechhours["Languages (ISO)"] = df_bloomspeechhours["ISO-639-3"].map(lambda x: langcodes.standardize_tag(x, macro=True))

In [ ]:
# Preprocess from raw data (once only)
# df_fleursspeechhours = pd.read_csv("speech_supportingdata/fleurs_splithoursorig.csv")
# df_fleursspeechhours = df_fleursspeechhours.groupby("language")["num_samples"].sum().reset_index()
# df_fleursspeechhours["Hours"] = df_fleursspeechhours["num_samples"] / 16000 / 60 / 60
# df_fleursspeechhours.columns = ["Languages (ISO)", *df_fleursspeechhours.columns[1:]]

# df_fleursspeechhours.to_csv("speech_supportingdata/fleurs_splithours.csv", index=False)


df_fleursspeechhours = pd.read_csv("data/speech_supporting_data/fleurs_splithours.csv")

In [ ]:
dataset_hoursmapping = {
    "yodas": df_yodashours,
    "common-voice-corpus-170": df_commonvoicehours,
    "multilingual-librispeech": df_multilinguallibrispeechhours,
    "bloom-speech": df_bloomspeechhours,
    "fleurs": df_fleursspeechhours
}

special_cases = {
    "yodas": {
        "sr-Latn": "sr", # YODAS metadata doesn't specify the script
        "he": "iw" # YODAS appears to use the old ISO639-1 code
    },
    "fleurs": {
        "no": "nb" # FLEURS specifies locale code
    }
}

def get_hours_for_dataset_and_language(row: pd.Series) -> float:
    dataset = row["Unique Dataset Identifier"]
    language = row["Languages (ISO)"]
    language = langcodes.standardize_tag(language)

    if dataset in dataset_hoursmapping:
        if language in special_cases.get(dataset, {}):
            language = special_cases[dataset][language]

        hours_df = dataset_hoursmapping[dataset]
        hours = hours_df[hours_df["Languages (ISO)"] == language]["Hours"].sum()
        if hours == 0:
            print("Hours not found for language code %s in dataset %s" % (language, dataset))
        return hours

    return row["Hours"]

## Gini Coefficient Across Languages by (Cumulative) Total Hours

In [ ]:
lang_codes_to_families = {}
lang_codes_to_names = {}
iso_codes_to_langs = {}
lang_families = {}

df_langsglottolog = pd.read_csv("data/speech_supporting_data/languages.csv")
for i, row in df_langsglottolog.iterrows():
    lang_id = row["ID"]

    iso_code = row["Closest_ISO369P3code"]

    iso_codes_to_langs[iso_code] = lang_id
    lang_codes_to_names[lang_id] = row["Name"]

    if row["Level"] == "family":
        lang_families[lang_id] = row["Name"]

    if pd.isna(iso_code):
        continue

    family_id = row["Family_ID"]
    if pd.isna(family_id):
        continue

    lang_codes_to_families[lang_id] = family_id


def get_langfamily(lang: str) -> str:
    lang = lang.split("-")[0].split("_")[0]
    try:
        # Need to iteratively seek to the top level
        lang = langcodes.get(langcodes.standardize_tag(lang, macro=True)).to_alpha3()
        lang = iso_codes_to_langs.get(lang, lang)

        while lang in lang_codes_to_families:
            lang = lang_codes_to_families[lang]

        lang = lang_families[lang]

    except:
        lang = "Other"

    return lang

In [ ]:
df_speechlanguagesn = df_speechlanguagenames.copy()

df_speechlanguagesn["Languages (ISO)"] = df_speechlanguagesn["Languages (ISO)"].map(lambda x : x.split("_")[0].split("-")[0])
df_speechlanguagesn["Language Family"] = df_speechlanguagesn["Languages (ISO)"].map(get_langfamily)

# Subdivide hours evenly across the languages given in each dataset
df_speechlanguagesn["Hours"] = df_speechlanguagesn.groupby(["Unique Dataset Identifier", "Languages (ISO)"])["Hours"].transform(
    lambda x: x / x.count()
)
df_speechlanguagesn["Hours"] = df_speechlanguagesn.apply(get_hours_for_dataset_and_language, axis=1)

df_speechlanguagesn = df_speechlanguagesn.sort_values(by="Year Released")

In [ ]:
# Ensure that, for each of those datasets, we have heterogenous language hours
for dataset in ["yodas", "common-voice-corpus-170", "multilingual-librispeech", "bloom-speech", "fleurs"]:
    assert df_speechlanguagesn[
            df_speechlanguagesn["Unique Dataset Identifier"] == dataset
        ]["Hours"].nunique() > 1, "Dataset %s has homogenous language hours" % dataset

In [ ]:
# Gini coefficient for hours across languages
speechlanguages_totalhours = df_speechlanguagesn.groupby("Languages (ISO)")["Hours"].sum().reset_index(name="Total Hours")

multimodal_util.gini(speechlanguages_totalhours["Total Hours"].values)

In [ ]:
# Gini coefficient for hours across language-families
speechlanguagesf_totalhours = df_speechlanguagesn.groupby("Language Family")["Hours"].sum().reset_index(name="Total Hours")

multimodal_util.gini(speechlanguagesf_totalhours["Total Hours"].values)

In [ ]:
# Get the cumulative hours by language over time
df_speechlanguagescumulativehours = df_speechlanguagesn.groupby(
    ["Year Released", "Languages (ISO)"]
)["Hours"].sum().groupby(
    "Languages (ISO)"
).cumsum().reset_index(name="Cumulative Hours")

# Calculate Gini coefficient and CIs for cumulative hours by language
df_speechlanguagescumulativehoursgini = df_speechlanguagescumulativehours.groupby(
    "Year Released"
)["Cumulative Hours"].apply(
    lambda x: multimodal_util.bootstrap_cis_for_gini(x.values)
).reset_index(
    name="Gini"
)

df_speechlanguagescumulativehoursgini["Gini Mean"] = df_speechlanguagescumulativehoursgini["Gini"].map(lambda x: x[0])
df_speechlanguagescumulativehoursgini["Gini Lower"] = df_speechlanguagescumulativehoursgini["Gini"].map(lambda x: x[1])
df_speechlanguagescumulativehoursgini["Gini Upper"] = df_speechlanguagescumulativehoursgini["Gini"].map(lambda x: x[2])

# Get the cumulative hours by language-family over time
df_speechlanguagesfamilycumulativehours = df_speechlanguagesn.groupby(
    ["Year Released", "Language Family"]
)["Hours"].sum().groupby(
    "Language Family"
).cumsum().reset_index(name="Cumulative Hours")

# Calculate Gini coefficient and CIs for cumulative hours by language
df_speechlanguagesfamilycumulativehoursgini = df_speechlanguagesfamilycumulativehours.groupby(
    "Year Released"
)["Cumulative Hours"].apply(
    lambda x: multimodal_util.bootstrap_cis_for_gini(x.values)
).reset_index(
    name="Gini"
)

df_speechlanguagesfamilycumulativehoursgini["Gini Mean"] = df_speechlanguagesfamilycumulativehoursgini["Gini"].map(lambda x: x[0])
df_speechlanguagesfamilycumulativehoursgini["Gini Lower"] = df_speechlanguagesfamilycumulativehoursgini["Gini"].map(lambda x: x[1])
df_speechlanguagesfamilycumulativehoursgini["Gini Upper"] = df_speechlanguagesfamilycumulativehoursgini["Gini"].map(lambda x: x[2])

df_speechlanguagescumulativehoursgini["Type"] = "Languages"
df_speechlanguagesfamilycumulativehoursgini["Type"] = "Families"

df_speechlanguagesfamilycumulativehoursgini = pd.concat(
    [df_speechlanguagescumulativehoursgini, df_speechlanguagesfamilycumulativehoursgini]
)

chart_meanlangf = alt.Chart(
    df_speechlanguagesfamilycumulativehoursgini
).mark_line().encode(
    x=alt.X(
        "Year Released:N",
        title="",
        sort=YEARS_ORDER,
        axis=alt.Axis(labelAngle=-30),
        scale=alt.Scale(padding=0)
    ),
    y=alt.Y(
        "Gini Mean:Q",
        title="Gini (Cumulative)",
        scale=alt.Scale(zero=False)
    ),
    color=alt.Color(
        "Type:N",
        title="Type",
        scale=alt.Scale(
            domain=["Languages", "Families"],
            range=["#82b5cf", "#ff7fde"]
        )
    )
)

chart_meanpointslangf = alt.Chart(
    df_speechlanguagesfamilycumulativehoursgini
).mark_point().encode(
    x=alt.X(
        "Year Released:N",
        title="",
        sort=YEARS_ORDER,
        axis=alt.Axis(labelAngle=-30),
        scale=alt.Scale(padding=0)
    ),
    y=alt.Y(
        "Gini Mean:Q",
        title="Gini (Cumulative)",
        scale=alt.Scale(zero=False)
    ),
    color="Type:N"
)

chart_cislangf = alt.Chart(
    df_speechlanguagesfamilycumulativehoursgini
).mark_area(
    opacity=0.2
).encode(
    x=alt.X(
        "Year Released:N",
        title="",
        sort=YEARS_ORDER,
        axis=alt.Axis(labelAngle=-30)
    ),
    y="Gini Lower:Q",
    y2="Gini Upper:Q",
    color="Type:N"
)

chart_langf = (chart_cislangf + chart_meanlangf + chart_meanpointslangf).configure_axis(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    grid=False
).properties(
    width=600,
    height=200,
    title="For %d Languages (ISO) and %d Language Families" % (
        df_speechlanguagesn["Languages (ISO)"].nunique(),
        df_speechlanguagesn["Language Family"].nunique()
    )
)

if PLOT_TOFILE:
    chart_langf.save(
        os.path.join(PLOT_DIR, "speech_languagefamilies-giniyears.png"),
        ppi=PLOT_PPI
    )

chart_langf

In [ ]:
df_speechlanguagesn["First Year Released"] = df_speechlanguagesn.groupby("Languages (ISO)")["Year Released"].transform("min")
df_speechlanguagesn = df_speechlanguagesn.drop_duplicates(subset=["Languages (ISO)"])

chart_langcount = alt.Chart(
    df_speechlanguagesn.groupby("First Year Released")["Languages (ISO)"].count().cumsum().reset_index(name="Total Languages")
).mark_bar().encode(
    x=alt.X(
        "First Year Released:N",
        title="",
        sort=YEARS_ORDER,
        axis=alt.Axis(labelAngle=-30)
    ),
    y=alt.Y(
        "Total Languages:Q",
        title="Total Languages"
    ),
    color=alt.value("#82b5cf")
)

if PLOT_TOFILE:
    chart_langcount.save(
        os.path.join(PLOT_DIR, "speech_languages-years.png"),
        ppi=PLOT_PPI
    )

chart_langcount

In [ ]:
chart_multilingualdatasets = alt.Chart(df_speech).mark_bar().encode(
    x=alt.X(
        "Year Released:N",
        title="Year Released",
        sort=YEARS_ORDER,
        axis=alt.Axis(labelAngle=-30)
    ),
    y=alt.Y(
        "Languages (Count)",
        title="Languages (Count)",
        scale=alt.Scale(
            domain=[1, 199]
        )
    ),
    color=alt.Color(
        "Unique Dataset Identifier:O",
        legend=None
    )
)

chart_multilingualdatasets

In [ ]:
chart_langcombined = alt.hconcat(
    chart_langcount.properties(
        width=400,
        height=200,
        title="A"
    ),
    chart_multilingualdatasets.properties(
        width=400,
        height=200,
        title="B"
    ),
    (chart_cislangf + chart_meanlangf + chart_meanpointslangf).properties(
        width=400,
        height=200,
        title="C"
    ),
    padding=0
).configure_axis(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    grid=False
).configure_legend(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    orient="bottom-right",
    labelLimit=MAX_LABELLIMIT
).configure_header(
    titleFontSize=FONT_SIZE,
    labelFontSize=FONT_SIZE
).configure_title(
    fontSize=FONT_SIZE,
    anchor="start"
).resolve_scale(
    x="independent",
    y="independent",
    color="independent"
)

if PLOT_TOFILE:
    chart_langcombined.save(
        os.path.join(PLOT_DIR, "speech_languages-yearscombinedgini.pdf"),
        ppi=PLOT_PPI
    )

chart_langcombined

## Hours by Language Family

In [ ]:
df_speechlanguagesf = speechlanguagesf_totalhours.copy()
topk_langf = df_speechlanguagesf.sort_values(by="Total Hours", ascending=False).head(10)["Language Family"]
df_speechlanguagesf["Language Family"] = df_speechlanguagesf["Language Family"].map(
    lambda x: x if x in topk_langf.values else "Other"
)

In [ ]:
alt.Chart(df_speechlanguagesf).mark_bar().encode(
    x=alt.X(
        "Language Family:N",
        title="Language Family",
        sort=[item for item in topk_langf.tolist() if item != "Other"] + ["Other"],
        axis=alt.Axis(labelAngle=-30)
    ),
    y=alt.Y(
        "Total Hours:Q",
        title="Hours",
        axis=alt.Axis(format="~s")
    )
).configure_axis(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    grid=False
).properties(
    width=600,
    height=200
)

## Source Category by Year

In [ ]:
INCLUDE_TOP_N_CATEGORIES = 6
df_speechsourceyears = df_speech.explode("Source Category")
df_speechsourceyears = multimodal_util.reduce_categories_to_topk(df_speechsourceyears, "Source Category", INCLUDE_TOP_N_CATEGORIES)

df_speechsourceyears = df_speechsourceyears.sort_values(by="Year Released")

In [ ]:
base_sourceyear = alt.Chart(
    df_speechsourceyears
).mark_bar().encode(
    x=alt.X(
        "Year Released:N",
        title="Year Released",
        sort=YEARS_ORDER,
        axis=alt.Axis(labelAngle=-30)
    ),
    y=alt.Y(
        "count():Q",
        stack="normalize",
        axis=alt.Axis(format="%"),
        title="Pct. Datasets"
    ),
    color=alt.Color(
        "Source Category:N",
        title="Source Category"
    )
).properties(
    width=600,
    height=160
)

text_sourceyear = alt.Chart(df_speechsourceyears).mark_text(
    dy=-90,
    align="center",
    baseline="top",
    fontSize=12
).encode(
    x=alt.X(
        "Year Released:N",
        title="Year Released",
        sort=YEARS_ORDER
    ),
    text="count():Q"
)

chart_sourceyear = (base_sourceyear + text_sourceyear).configure_axis(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE
).configure_legend(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    orient=LEGEND_POSITION,
    columns=4,
    labelLimit=MAX_LABELLIMIT
)


if PLOT_TOFILE:
    chart_sourceyear.save(
        os.path.join(PLOT_DIR, "speech_sourcecategories-years.png"),
        ppi=PLOT_PPI
    )

chart_sourceyear

## Total Hours by Source Category (Cumulative)

In [ ]:
INCLUDE_TOP_N_CATEGORIES = 6

df_speechsourceyears = df_speech.explode("Source Category")
df_speechsourceyears = multimodal_util.reduce_categories_to_topk(df_speechsourceyears, "Source Category", INCLUDE_TOP_N_CATEGORIES)

df_speechsourceyearscumulativehours = df_speechsourceyears.groupby(
    ["Year Released", "Source Category"]
)["Hours"].sum().groupby(
    "Source Category"
).cumsum().reset_index(name="Cumulative Hours")

df_speechsourceyearscumulativehours = df_speechsourceyearscumulativehours.sort_values(by="Year Released")

In [ ]:
chart_sourceyearhours = alt.Chart(
    df_speechsourceyearscumulativehours
).mark_line().encode(
    x=alt.X(
        "Year Released:N",
        title="Year Released",
        sort=YEARS_ORDER,
        axis=alt.Axis(labelAngle=-30)
    ),
    y=alt.Y(
        "Cumulative Hours:Q",
        title="Cumulative Hours",
        scale=alt.Scale(
            type="symlog",
            constant=1000,
            domain=[1, 1000000]
        ),
        axis=alt.Axis(
            values=[0, 1, 1000, 10000, 100000, 1000000],
            labelExpr="datum.value >= 1000000 ? datum.value / 1000000 + 'M' : datum.value >= 1000 ? datum.value / 1000 + 'K' : datum.value"
        )
    ),
    color=alt.Color(
        "Source Category:N",
        title="Source Category"
    )
).properties(
    width=400,
    height=160
)

if PLOT_TOFILE:
    chart_sourceyearhours.save(
        os.path.join(PLOT_DIR, "speech_sourcecategories-cumulativehours.png"),
        ppi=PLOT_PPI
    )

chart_sourceyearhours

#### Combine Source Years-Based Plots

In [ ]:
chart_sourceyearcombined = alt.hconcat(
    base_sourceyear,
    chart_sourceyearhours
).configure_axis(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    grid=False
).configure_legend(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    labelLimit=MAX_LABELLIMIT
).configure_header(
    titleFontSize=FONT_SIZE,
    labelFontSize=FONT_SIZE
).configure_title(
    fontSize=FONT_SIZE
).resolve_scale(
    x="independent",
    y="independent"
)

if PLOT_TOFILE:
    chart_sourceyearcombined.save(
        os.path.join(PLOT_DIR, "speech_sourcecategories-yearscombined.png"),
        ppi=PLOT_PPI
    )

chart_sourceyearcombined

## Creator Categories by Year

Note: we use the original annotations here instead of the DPI constants, for a different view.

In [ ]:
df_speechcategoriesyears = df_speech.explode("Creator Categories")
df_speechcategoriesyears = df_speechcategoriesyears.sort_values(by="Year Released")

In [ ]:
base = alt.Chart(
    df_speechcategoriesyears
).mark_bar().encode(
    x=alt.X(
        "Year Released:N",
        title="Year Released",
        sort=YEARS_ORDER,
        axis=alt.Axis(labelAngle=-30)
    ),
    y=alt.Y(
        "count():Q",
        stack="normalize",
        axis=alt.Axis(format="%"),
        title="Pct. Datasets"
    ),
    color=alt.Color(
        "Creator Categories:N",
        title="Creator Category"
    )
).properties(
    width=600,
    height=100
)

text = alt.Chart(df_speechsourceyears).mark_text(
    dy=-68,
    align="center",
    baseline="top",
    fontSize=12
).encode(
    x=alt.X(
        "Year Released:N",
        title="Year Released",
        sort=YEARS_ORDER
    ),
    text="count():Q"
)

chart = (base + text).configure_axis(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE
).configure_legend(
    labelFontSize=FONT_SIZE,
    titleFontSize=FONT_SIZE,
    orient=LEGEND_POSITION,
    columns=4,
    labelLimit=MAX_LABELLIMIT
)

if PLOT_TOFILE:
    chart.save(
        os.path.join(PLOT_DIR, "speech_categories-years.png"),
        ppi=PLOT_PPI
    )

chart

## Summary Tables
#### Table of License Type

In [ ]:
licensetype_counts = df_speech["License Type"].value_counts()
df_licensetypes = pd.concat([
    licensetype_counts,
    (licensetype_counts / licensetype_counts.sum()).round(4) * 100
], axis=1)

df_licensetypes.columns = ["Count", "Pct."]

df_licensetypes

#### Tables of YouTube-Sourced Portions (By Count and Hour, Per License-Type)

In [ ]:
# By count
df_speech["YouTube"] = df_speech["Source"].map(
    lambda x: "YouTube" if any("youtube" in xi.lower() for xi in x) else "Other"
)

df_youtube = df_speech.groupby(["License Type", "YouTube"]).size().reset_index(name="Count")
df_youtube = df_youtube.sort_values(by="Count")
df_youtube["Pct."] = df_youtube.groupby("License Type")["Count"].transform(lambda x: (x / x.sum()).round(4) * 100)

df_youtube

In [ ]:
# By hours
df_youtubehours = df_speech.groupby(["License Type", "YouTube"])["Hours"].sum().reset_index(name="Total Hours")
df_youtubehours = df_youtubehours.sort_values(by="Total Hours")
df_youtubehours["Pct."] = df_youtubehours.groupby("License Type")["Total Hours"].transform(lambda x: (x / x.sum()).round(4) * 100)

df_youtubehours

#### Tables of Creator Categories (By Count and Hour)

In [ ]:
df_creatorcategories = df_speech.explode("Creator Categories").groupby("Creator Categories").size().reset_index(name="Count")
df_creatorcategories["Pct."] = df_creatorcategories["Count"].transform(lambda x: (x / x.sum()).round(4) * 100)

df_creatorcategories

In [ ]:
df_creatorcategories = df_speech.explode("Creator Categories").groupby("Creator Categories")["Hours"].sum().reset_index(name="Total Hours")
df_creatorcategories["Pct."] = df_creatorcategories["Total Hours"].transform(lambda x: (x / x.sum()).round(4) * 100)

df_creatorcategories